In [1]:
import requests as r
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json
from threading import Thread

# selenium

## get the links of movies different genres

In [ ]:
links = []

In [484]:
def get_genre(genre):
    root_url = "https://www.imdb.com/search/title/?title_type=feature&genres=%s"%genre.lower()
    driver = webdriver.Firefox()
    wait = WebDriverWait(driver, 5, 0.5)
    driver.get(root_url)
    driver.implicitly_wait(10)
    print("Fetching %s"%genre)
    for _ in range(10):
        button = driver.find_element(by = By.XPATH, value = "/html/body/div[2]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/div[2]/div/span/button")
        driver.execute_script("arguments[0].scrollIntoView();", button)
        time.sleep(1)
        button.click()
        time.sleep(2)
        wait.until(EC.visibility_of(button))
    soup = BeautifulSoup(driver.page_source)
    table = soup.find("main").find_all("div")[1].find_all("div", attrs = {"class": "ipc-page-content-container ipc-page-content-container--center"})[1].find("section").find("section").find("div").find("div", attrs = {"role": "tabpanel"}).find("div").find("section").find("div", attrs = {"class": "ipc-page-grid ipc-page-grid--bias-left ipc-page-grid__item ipc-page-grid__item--span-2"}).find("div", attrs = {"class": "ipc-page-grid__item ipc-page-grid__item--span-2"}).find("ul").find_all("li")
    imdb_url = "https://www.imdb.com/"
    for li in table:
        link = li.find("div").find("div").find("div").find("div").find_all("div")[1].find("a").get("href")
        links.append(imdb_url + link)

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}


In [ ]:
genres = ['Action',
 'Drama',
 'History',
 'Adventure',
 'Comedy',
 'War',
 'Romance',
 'Crime',
 'Fantasy',
 'Horror',
 'Mystery',
 'Thriller',
 'Documentary',
 'Biography',
 'Family',
 'Music',
 'Sci-Fi',
 'Animation',
 'Western',
 'Film-Noir',
 'Sport',
 'Musical',
 'News']
for genre in genres:
    thread = Thread(target = get_genre, daemon=True, args=(genre,))
    thread.start()

In [496]:
links = list(set(links))
with open("link_cache", "w") as f:
    json.dump(links, f)

## load links cache

In [3]:
with open("link_cache") as f:
    links = json.load(f)

# Requests

## fetch data from links

### multi-thread

In [4]:
links_list = []
for i in range(10):
    links_list.append(links[i*1474: (i+1)*1474])

In [39]:
html_cache = []
def get_rawdata(links, ind, start_ind):
    session = r.Session()
    try:
        for i in range(start_ind, len(links)):
            print("Fetching %i"%i)
            link = links[i]
            one_data = {}
            flag = 0
            for j in range(3):
                response = session.get(link, headers=headers)
                if response.status_code == 200:
                    flag = 1
                    break
                else:
                    print("Fail to fetch %s"%link)
                    print("Retry for %i time"%(j+1))
                    time.sleep(10)
            if flag:
                html_cache.append(response.text)
                time.sleep(1)
        thread_pool[ind] = i
    except:
        thread_pool[ind] = i

In [40]:
def get_data(links, ind, start_ind):
    session = r.Session()
    try:
        for i in range(start_ind, len(links)):
            print("Fetching %i"%i)
            link = links[i]
            one_data = {}
            flag = 0
            for j in range(3):
                response = session.get(link, headers=headers)
                if response.status_code == 200:
                    flag = 1
                    break
                else:
                    print("Fail to fetch %s"%link)
                    print("Retry for %i time"%(j+1))
                    time.sleep(10)
            if flag:
                None_count = 0
                soup = BeautifulSoup(response.text)
                try:
                    # title = soup.find("main").find("div").find("section").find("section").find_all("div")[6].find("section").find("section").find_all("div")[6].find("h1").find("span").text
                    title = soup.find("main").find("div").find("section").find("section").find("section", attrs = {"class": "ipc-page-section ipc-page-section--baseAlt ipc-page-section--tp-none ipc-page-section--bp-xs sc-e226b0e3-2 gelzOk"}).find("h1").find("span").text
                except:
                    title = None
                    None_count += 1
                    print("No title for %i"%i)

                try:
                    # year = soup.find("main").find("div").find("section").find("section").find_all("div")[6].find("section").find("section").find_all("div")[6].find("ul").find_all("li")[0].a.text
                    year = soup.find("main").find("div").find("section").find("section").find("section", attrs = {"class": "ipc-page-section ipc-page-section--baseAlt ipc-page-section--tp-none ipc-page-section--bp-xs sc-e226b0e3-2 gelzOk"}).find("ul", attrs = {"class": "ipc-inline-list ipc-inline-list--show-dividers sc-d8941411-2 cdJsTz baseAlt"}).a.text
                except:
                    year = None
                    None_count += 1
                    print("No year for %i"%i)
                
                try:
                    # length = soup.find("main").find("div").find("section").find("section").find_all("div")[6].find("section").find("section").find_all("div")[6].find("ul").find_all("li")[1].text
                    length = soup.find("main").find("div").find("section").find("section").find("section", attrs = {"class": "ipc-page-section ipc-page-section--baseAlt ipc-page-section--tp-none ipc-page-section--bp-xs sc-e226b0e3-2 gelzOk"}).find("ul", attrs = {"class": "ipc-inline-list ipc-inline-list--show-dividers sc-d8941411-2 cdJsTz baseAlt"}).find_all("li")[-1].text
                except:
                    length = None
                    None_count += 1
                    print("No length for %i"%i)

                try:
                    # imdb_rating = soup.find("main").find("div").find("section").find("section").find_all("div")[6].find("section").find("section").find_all("div")[8].find("div").find("div").find("span").find("div").find_all("div")[1].find("span").text
                    imdb_rating = soup.find("main").find("div").find("section").find("section").find("section", attrs = {"class": "ipc-page-section ipc-page-section--baseAlt ipc-page-section--tp-none ipc-page-section--bp-xs sc-e226b0e3-2 gelzOk"}).find("div", attrs = {"data-testid": "hero-rating-bar__aggregate-rating__score"}).span.text
                except:
                    imdb_rating = None
                    None_count += 1
                    print("No imdb_rating for %i"%i)

                try:        
                    # total_rating_person = soup.find("main").find("div").find("section").find("section").find_all("div")[6].find("section").find("section").find_all("div")[8].find("div").find("div").find("span").find("div").find_all("div")[-1].text
                    total_rating_person = soup.find("main").find("div").find("section").find("section").find("section", attrs = {"class": "ipc-page-section ipc-page-section--baseAlt ipc-page-section--tp-none ipc-page-section--bp-xs sc-e226b0e3-2 gelzOk"}).find("div", attrs = {"class": "sc-bde20123-3 gPVQxL"}).text
                except:
                    total_rating_person = None
                    None_count += 1
                    print("No total_rating_person for %i"%i)
                
                try:

                    # raw_genres = soup.find("main").find("div").find("section").find("section").find_all("div")[6].find("section").find("section").find_all("div")[31].find_all("div")[29].find_all("a")
                    raw_genres = soup.find("main").find("div").find("section").find("section").find("section", attrs = {"class": "ipc-page-section ipc-page-section--baseAlt ipc-page-section--tp-none ipc-page-section--bp-xs sc-e226b0e3-2 gelzOk"}).find("div", attrs = {"data-testid": "genres"}).find("div", attrs = {"class": "ipc-chip-list__scroller"}).find_all("a")
                    genres = [raw_genre.span.text for raw_genre in raw_genres]
                except:
                    genres = None
                    None_count += 1
                    print("No genres for %i"%i)
                
                try:
                    
                    # intro = soup.find("main").find("div").find("section").find("section").find_all("div")[6].find("section").find("section").find_all("div")[31].p.find_all("span")[1].text
                    intro = soup.find("main").find("div").find("section").find("section").find("section", attrs = {"class": "ipc-page-section ipc-page-section--baseAlt ipc-page-section--tp-none ipc-page-section--bp-xs sc-e226b0e3-2 gelzOk"}).find("span", attrs = {"data-testid": "plot-l"}).text
                except:
                    intro = None
                    None_count += 1
                    print("No intro for %i"%i)
                
                try:

                    raw_director = soup.find("main").find("div").find("section").find("section").find("section", attrs = {"class": "ipc-page-section ipc-page-section--baseAlt ipc-page-section--tp-none ipc-page-section--bp-xs sc-e226b0e3-2 gelzOk"}).find("ul", attrs = {"class": "ipc-metadata-list ipc-metadata-list--dividers-all title-pc-list ipc-metadata-list--baseAlt"}).find_all("li", attrs = {"class": "ipc-metadata-list__item"})[0]
                    directors = [i.a.text for i in raw_director.div.ul.find_all("li")]
                except:
                    directors = None
                    None_count += 1
                    print("No directors for %i"%i)
                
                try:
                    # raw_writers = soup.find("main").find("div").find("section").find("section").find_all("div")[6].find("section").find("section").find_all("div")[31].find_all("div")[32].find("ul").find_all("li", attrs = {"class": "ipc-metadata-list__item"})[1]
                    raw_writers = soup.find("main").find("div").find("section").find("section").find("section", attrs = {"class": "ipc-page-section ipc-page-section--baseAlt ipc-page-section--tp-none ipc-page-section--bp-xs sc-e226b0e3-2 gelzOk"}).find("ul", attrs = {"class": "ipc-metadata-list ipc-metadata-list--dividers-all title-pc-list ipc-metadata-list--baseAlt"}).find_all("li", attrs = {"class": "ipc-metadata-list__item"})[1]
                    writers = [i.a.text for i in raw_writers.div.ul.find_all("li")]
                except:
                    writers = None
                    None_count += 1
                    print("No writers for %i"%i)
                
                try:
                    # raw_stars = soup.find("main").find("div").find("section").find("section").find_all("div")[6].find("section").find("section").find_all("div")[31].find_all("div")[32].find("ul").find_all("li", attrs = {"class": "ipc-metadata-list__item"})[2]
                    raw_stars = soup.find("main").find("div").find("section").find("section").find("section", attrs = {"class": "ipc-page-section ipc-page-section--baseAlt ipc-page-section--tp-none ipc-page-section--bp-xs sc-e226b0e3-2 gelzOk"}).find("ul", attrs = {"class": "ipc-metadata-list ipc-metadata-list--dividers-all title-pc-list ipc-metadata-list--baseAlt"}).find_all("li", attrs = {"class": "ipc-metadata-list__item"})[2]
                    stars = [i.a.text for i in raw_stars.div.ul.find_all("li")]
                except:
                    stars = None
                    None_count += 1
                    print("No stars for %i"%i)
                if None_count == 10:
                    break
                one_data["title"] = title
                one_data["year"] = year
                one_data["length"] = length
                one_data["imdb_rating"] = imdb_rating
                one_data["total_rating_person"] = total_rating_person
                one_data["genres"] = genres
                one_data["intro"] = intro
                one_data["directors"] = directors
                one_data["writers"] = writers
                one_data["stars"] = stars
                one_data["link"] = link
                all_data.append(one_data)
                time.sleep(1)
        thread_pool[ind] = i
    except:
        thread_pool[ind] = i

In [ ]:
thread_pool = [0] * 10
all_data = []
for i in range(10):
    thread = Thread(target = get_data, args=(links_list[i], i, thread_pool[i], ), daemon=True)
    thread.start()
with open("data_cache", 'w') as f:
    json.dump(all_data, f)

## fetching the raw data

In [41]:
thread_pool = [0] * 10
html_cache = []
for i in range(10):
    thread = Thread(target = get_rawdata, args=(links_list[i], i, thread_pool[i], ), daemon=True)
    thread.start()
with open("rawdata_cache", 'w') as f:
    json.dump(html_cache, f)

Fetching 0
Fetching 0
Fetching 0
Fetching 0
Fetching 0
Fetching 0
Fetching 0
Fetching 0
Fetching 0
Fetching 0


Fetching 1
Fetching 1
Fetching 1
Fetching 1
Fetching 1
Fetching 1
Fetching 1
Fetching 1
Fetching 1
Fetching 1
Fetching 2
Fetching 2
Fetching 2
Fetching 2
Fetching 2
Fetching 2
Fetching 2
Fetching 2
Fetching 2
Fetching 2
Fetching 3
Fetching 3
Fetching 3
Fetching 3
Fetching 3
Fetching 3
Fetching 3
Fetching 3
Fetching 3
Fetching 3
Fetching 4
Fetching 4
Fetching 4
Fetching 4
Fetching 4
Fetching 4
Fetching 4
Fetching 4
Fetching 5
Fetching 4
Fetching 5
Fetching 5
Fetching 4
Fetching 5
Fetching 5
Fetching 5
Fetching 5
Fetching 5
Fetching 6
Fetching 6
Fetching 6
Fetching 6
Fetching 5
Fetching 6
Fetching 6
Fetching 6
Fetching 5
Fetching 6
Fetching 7
Fetching 7
Fetching 6
Fetching 7
Fetching 7
Fetching 7
Fetching 7Fetching 7

Fetching 6
Fetching 7
Fetching 8
Fetching 7
Fetching 8


## parsing the raw data

In [72]:
def raw_data_paser(raw_data):
    for i in range(len(raw_data)):
        print("Fetching %i"%i)
        data = raw_data[i]
        if 1:
            None_count = 0
            soup = BeautifulSoup(data)
            try:
                # title = soup.find("main").find("div").find("section").find("section").find_all("div")[6].find("section").find("section").find_all("div")[6].find("h1").find("span").text
                title = soup.find("main").find("div").find("section").find("section").find("section", attrs = {"class": "ipc-page-section ipc-page-section--baseAlt ipc-page-section--tp-none ipc-page-section--bp-xs sc-e226b0e3-2 gelzOk"}).find("h1").find("span").text
            except:
                title = None
                None_count += 1
                print("No title for %i"%i)

            try:
                # year = soup.find("main").find("div").find("section").find("section").find_all("div")[6].find("section").find("section").find_all("div")[6].find("ul").find_all("li")[0].a.text
                year = soup.find("main").find("div").find("section").find("section").find("section", attrs = {"class": "ipc-page-section ipc-page-section--baseAlt ipc-page-section--tp-none ipc-page-section--bp-xs sc-e226b0e3-2 gelzOk"}).find("ul", attrs = {"class": "ipc-inline-list ipc-inline-list--show-dividers sc-d8941411-2 cdJsTz baseAlt"}).a.text
            except:
                year = None
                None_count += 1
                print("No year for %i"%i)
            
            try:
                # length = soup.find("main").find("div").find("section").find("section").find_all("div")[6].find("section").find("section").find_all("div")[6].find("ul").find_all("li")[1].text
                length = soup.find("main").find("div").find("section").find("section").find("section", attrs = {"class": "ipc-page-section ipc-page-section--baseAlt ipc-page-section--tp-none ipc-page-section--bp-xs sc-e226b0e3-2 gelzOk"}).find("ul", attrs = {"class": "ipc-inline-list ipc-inline-list--show-dividers sc-d8941411-2 cdJsTz baseAlt"}).find_all("li")[-1].text
            except:
                length = None
                None_count += 1
                print("No length for %i"%i)

            try:
                # imdb_rating = soup.find("main").find("div").find("section").find("section").find_all("div")[6].find("section").find("section").find_all("div")[8].find("div").find("div").find("span").find("div").find_all("div")[1].find("span").text
                imdb_rating = soup.find("main").find("div").find("section").find("section").find("section", attrs = {"class": "ipc-page-section ipc-page-section--baseAlt ipc-page-section--tp-none ipc-page-section--bp-xs sc-e226b0e3-2 gelzOk"}).find("div", attrs = {"data-testid": "hero-rating-bar__aggregate-rating__score"}).span.text
            except:
                imdb_rating = None
                None_count += 1
                print("No imdb_rating for %i"%i)

            try:        
                # total_rating_person = soup.find("main").find("div").find("section").find("section").find_all("div")[6].find("section").find("section").find_all("div")[8].find("div").find("div").find("span").find("div").find_all("div")[-1].text
                total_rating_person = soup.find("main").find("div").find("section").find("section").find("section", attrs = {"class": "ipc-page-section ipc-page-section--baseAlt ipc-page-section--tp-none ipc-page-section--bp-xs sc-e226b0e3-2 gelzOk"}).find("div", attrs = {"class": "sc-bde20123-3 gPVQxL"}).text
            except:
                total_rating_person = None
                None_count += 1
                print("No total_rating_person for %i"%i)
            
            try:

                # raw_genres = soup.find("main").find("div").find("section").find("section").find_all("div")[6].find("section").find("section").find_all("div")[31].find_all("div")[29].find_all("a")
                raw_genres = soup.find("main").find("div").find("section").find("section").find("section", attrs = {"class": "ipc-page-section ipc-page-section--baseAlt ipc-page-section--tp-none ipc-page-section--bp-xs sc-e226b0e3-2 gelzOk"}).find("div", attrs = {"data-testid": "genres"}).find("div", attrs = {"class": "ipc-chip-list__scroller"}).find_all("a")
                genres = [raw_genre.span.text for raw_genre in raw_genres]
            except:
                genres = None
                None_count += 1
                print("No genres for %i"%i)
            
            try:
                
                # intro = soup.find("main").find("div").find("section").find("section").find_all("div")[6].find("section").find("section").find_all("div")[31].p.find_all("span")[1].text
                intro = soup.find("main").find("div").find("section").find("section").find("section", attrs = {"class": "ipc-page-section ipc-page-section--baseAlt ipc-page-section--tp-none ipc-page-section--bp-xs sc-e226b0e3-2 gelzOk"}).find("span", attrs = {"data-testid": "plot-l"}).text
            except:
                intro = None
                None_count += 1
                print("No intro for %i"%i)
            
            try:

                raw_director = soup.find("main").find("div").find("section").find("section").find("section", attrs = {"class": "ipc-page-section ipc-page-section--baseAlt ipc-page-section--tp-none ipc-page-section--bp-xs sc-e226b0e3-2 gelzOk"}).find("ul", attrs = {"class": "ipc-metadata-list ipc-metadata-list--dividers-all title-pc-list ipc-metadata-list--baseAlt"}).find_all("li", attrs = {"class": "ipc-metadata-list__item"})[0]
                directors = [i.a.text for i in raw_director.div.ul.find_all("li")]
            except:
                directors = None
                None_count += 1
                print("No directors for %i"%i)
            
            try:
                # raw_writers = soup.find("main").find("div").find("section").find("section").find_all("div")[6].find("section").find("section").find_all("div")[31].find_all("div")[32].find("ul").find_all("li", attrs = {"class": "ipc-metadata-list__item"})[1]
                raw_writers = soup.find("main").find("div").find("section").find("section").find("section", attrs = {"class": "ipc-page-section ipc-page-section--baseAlt ipc-page-section--tp-none ipc-page-section--bp-xs sc-e226b0e3-2 gelzOk"}).find("ul", attrs = {"class": "ipc-metadata-list ipc-metadata-list--dividers-all title-pc-list ipc-metadata-list--baseAlt"}).find_all("li", attrs = {"class": "ipc-metadata-list__item"})[1]
                writers = [i.a.text for i in raw_writers.div.ul.find_all("li")]
            except:
                writers = None
                None_count += 1
                print("No writers for %i"%i)
            
            try:
                # raw_stars = soup.find("main").find("div").find("section").find("section").find_all("div")[6].find("section").find("section").find_all("div")[31].find_all("div")[32].find("ul").find_all("li", attrs = {"class": "ipc-metadata-list__item"})[2]
                raw_stars = soup.find("main").find("div").find("section").find("section").find("section", attrs = {"class": "ipc-page-section ipc-page-section--baseAlt ipc-page-section--tp-none ipc-page-section--bp-xs sc-e226b0e3-2 gelzOk"}).find("ul", attrs = {"class": "ipc-metadata-list ipc-metadata-list--dividers-all title-pc-list ipc-metadata-list--baseAlt"}).find_all("li", attrs = {"class": "ipc-metadata-list__item"})[2]
                stars = [i.a.text for i in raw_stars.div.ul.find_all("li")]
            except:
                stars = None
                None_count += 1
                print("No stars for %i"%i)
            if None_count == 10:
                break
            one_data = {}
            one_data["title"] = title
            one_data["year"] = year
            one_data["length"] = length
            one_data["imdb_rating"] = imdb_rating
            one_data["total_rating_person"] = total_rating_person
            one_data["genres"] = genres
            one_data["intro"] = intro
            one_data["directors"] = directors
            if not stars and writers:
                one_data["writers"] = None
                one_data["stars"] = writers
            else:
                one_data["writers"] = writers
                one_data["stars"] = stars
            all_data.append(one_data)


## parsing the raw data

In [73]:
all_data = []
raw_data_paser(html_cache)
with open("data_cache", 'w') as f:
    json.dump(all_data, f)

Fetching 0
No year for 0
No length for 0
No imdb_rating for 0
No total_rating_person for 0
No intro for 0
No directors for 0
No writers for 0
No stars for 0
Fetching 1
Fetching 2
Fetching 3
Fetching 4
Fetching 5
Fetching 6
Fetching 7
Fetching 8
Fetching 9
Fetching 10
No stars for 10
Fetching 11
Fetching 12
Fetching 13
Fetching 14
Fetching 15
Fetching 16
Fetching 17
Fetching 18
Fetching 19
Fetching 20
Fetching 21
Fetching 22
Fetching 23
Fetching 24
Fetching 25
Fetching 26
No year for 26
No length for 26
No imdb_rating for 26
No total_rating_person for 26
Fetching 27
Fetching 28
Fetching 29
Fetching 30
Fetching 31
Fetching 32
Fetching 33
Fetching 34
Fetching 35
Fetching 36
Fetching 37
Fetching 38
Fetching 39
Fetching 40
Fetching 41
Fetching 42
Fetching 43
Fetching 44
Fetching 45
Fetching 46
Fetching 47
Fetching 48
Fetching 49
Fetching 50
Fetching 51
No stars for 51
Fetching 52
Fetching 53
Fetching 54
Fetching 55
Fetching 56
Fetching 57
Fetching 58
Fetching 59
Fetching 60
Fetching 61
Fetc

In [ ]:
with open("data_cache", 'w') as f:
    json.dump(all_data, f)

## cache

In [64]:
# with open("rawdata_cache", 'w') as f:
#     json.dump(html_cache, f)

In [14]:
with open("data_cache", 'w') as f:
    json.dump(all_data, f)

## alternative way to get stars

In [20]:
response = r.get("https://www.imdb.com/title/tt13751694/?ref_=sr_t_1", headers = headers)

In [21]:
soup = BeautifulSoup(response.text)

In [36]:
raw_stars = soup.main.find("div", attrs={"class": "ipc-page-grid ipc-page-grid--bias-left"}).find("section", attrs = {"data-testid": "title-cast"}).find("div", attrs={"class": "ipc-sub-grid ipc-sub-grid--page-span-2 ipc-sub-grid--wraps-at-above-l ipc-shoveler__grid"}).find_all("div", attrs = {"data-testid": "title-cast-item"})
stars = [i.find("a", attrs = {"data-testid": "title-cast-item__actor"}).text for i in raw_stars]